**Import library**

In [1]:
from openai import AzureOpenAI
import pyodbc
import pandas as pd

import os
from dotenv import load_dotenv
from pathlib import Path

import re
from datetime import datetime

**Locate env path**

In [2]:
script_dir = Path(os.getcwd())
env_path = script_dir / '.env'
load_dotenv(env_path)

print(f'Script Path:\t{script_dir}\nenv Path:\t{env_path}\nLoad env:\t{load_dotenv(env_path)}')

Script Path:	c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester
env Path:	c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester\.env
Load env:	True


In [3]:
server = os.getenv('azure_server')
database = os.getenv('azure_database')
username = os.getenv('azure_username')
password = os.getenv('azure_password')

**Connection Database**

In [4]:
conn_str = f"""Driver={{ODBC Driver 17 for SQL Server}}; Server={server}; Database={database}; Uid={username}; Pwd={password}; Encrypt=yes; TrustServerCertificate=no; Connection Timeout=30;"""

In [5]:
conn = pyodbc.connect(conn_str)
conn

**Ai Prompt**

In [6]:
# Path to your Schema.sql file
sql_file_path = script_dir / "sql_script" / "Schema.sql"

# Read the file content into a Python string
with open(sql_file_path, 'r', encoding='utf-8') as file:
    schema_query = file.read()

print(schema_query)

-- =============================================
-- Author:		TANAKRIT-THONGPENG
-- Create date: 27/06/2025
-- Description:	SCHEMA QUERY
-- =============================================

SELECT 
    TABLE_CATALOG, 
    TABLE_SCHEMA, 
    TABLE_NAME, 
    COLUMN_NAME, 
    DATA_TYPE
FROM 
    INFORMATION_SCHEMA.COLUMNS
ORDER BY 
    TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;


In [7]:
database = 'SalesLT'
schema = 'dbo'

schema_df = pd.read_sql_query(schema_query, conn)
schema_df = schema_df[schema_df['TABLE_CATALOG'] == f'{database}']
schema_df = schema_df[schema_df['TABLE_SCHEMA'] == f'{schema}']

schema_text = ""
for (schema, table), group in schema_df.groupby(['TABLE_SCHEMA', 'TABLE_NAME']):
    columns = ", ".join(f"{row.COLUMN_NAME} ({row.DATA_TYPE})" for _, row in group.iterrows())
    schema_text += f"Table {schema}.{table}: {columns}\n"


C:\Users\Admins\AppData\Local\Temp\ipykernel_11092\3743000493.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql_query(schema_query, conn)


In [8]:
schema_df

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE
0,SalesLT,dbo,Address,AddressID,int
1,SalesLT,dbo,Address,AddressLine1,nvarchar
2,SalesLT,dbo,Address,AddressLine2,nvarchar
3,SalesLT,dbo,Address,City,nvarchar
4,SalesLT,dbo,Address,StateProvince,nvarchar
...,...,...,...,...,...
94,SalesLT,dbo,SalesOrderHeader,Freight,money
95,SalesLT,dbo,SalesOrderHeader,TotalDue,money
96,SalesLT,dbo,SalesOrderHeader,Comment,nvarchar
97,SalesLT,dbo,SalesOrderHeader,rowguid,uniqueidentifier


In [9]:
user_prompt = "Can you add more table, The table should have at least 1 field that keeping json format as a nvarchar and should be related to the existing tables in the schema."

In [10]:
model_name = os.getenv("openai_model_name")

client = AzureOpenAI(
    api_key=os.getenv("openai_model_api_key"),
    api_version=os.getenv("openai_api_version"),
    azure_endpoint=os.getenv("openai_api_endpoint")
)

In [11]:
system_message = f"""
You are a helpful assistant that writes SQL Server queries.
Here is the database schema:

{schema_text}
"""

In [12]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Write a SQL Server query for this: {user_prompt}"}
    ]
)

**Generate Output**

In [13]:
generated_sql = response.choices[0].message.content
print("Generated SQL:\n", generated_sql)

Generated SQL:
 Sure! Here is an example of a new table called `ProductTag` that includes a field named `TagData` storing JSON format data and is related to the `Product` table:

```sql
CREATE TABLE dbo.ProductTag (
    ProductTagID int PRIMARY KEY,
    ProductID int,
    TagData nvarchar(max),
    CONSTRAINT FK_ProductTag_ProductID FOREIGN KEY (ProductID) REFERENCES dbo.Product(ProductID)
);
```

In this hypothetical example, each product in the `Product` table can have multiple tags stored as JSON data in the `ProductTag` table. You can modify the table structure and relationships based on your specific requirements and relationships with existing tables in the schema.


In [14]:
# Extract just the SQL code between ```sql and ```
sql_code_blocks = re.findall(r"```sql\s*(.*?)```", response.choices[0].message.content, re.DOTALL)
description_blocks = re.split(r"```(?:sql)?[\s\S]*?```", response.choices[0].message.content)

if sql_code_blocks:
    generated_sql = sql_code_blocks[0].strip()
    generated_description = description_blocks[1].strip()
else:
    generated_sql = response.choices[0].message.content.strip()  # fallback
    generated_description = response.choices[0].message.content.strip()  # fallback

print(f"{generated_sql}\n\n{generated_description}")

CREATE TABLE dbo.ProductTag (
    ProductTagID int PRIMARY KEY,
    ProductID int,
    TagData nvarchar(max),
    CONSTRAINT FK_ProductTag_ProductID FOREIGN KEY (ProductID) REFERENCES dbo.Product(ProductID)
);

In this hypothetical example, each product in the `Product` table can have multiple tags stored as JSON data in the `ProductTag` table. You can modify the table structure and relationships based on your specific requirements and relationships with existing tables in the schema.


In [15]:
query_string = f"""{generated_sql}"""
query_string

'CREATE TABLE dbo.ProductTag (\n    ProductTagID int PRIMARY KEY,\n    ProductID int,\n    TagData nvarchar(max),\n    CONSTRAINT FK_ProductTag_ProductID FOREIGN KEY (ProductID) REFERENCES dbo.Product(ProductID)\n);'

**Final Dataframe**

In [16]:
try:
    df = pd.read_sql_query(query_string, conn)
    print("Query Results:")
    display(df)
except Exception as e:
    print("Error running the query:", e)

Error running the query: 'NoneType' object is not iterable


C:\Users\Admins\AppData\Local\Temp\ipykernel_11092\1492314482.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_string, conn)


**Create SQL Script from Output**

In [17]:
Author = "AI Generated SQL Query"

day = datetime.now().day
month = datetime.now().month
year = datetime.now().year
date_obj = datetime(year, month, day)
create_date = date_obj.strftime("%d/%m/%Y")

description = generated_description


In [18]:
header = f"""-- =============================================
-- Author:		{Author}
-- Create date: {create_date}
-- Description:	{description}
-- =============================================
"""

In [19]:
header

'-- =============================================\n-- Author:\t\tAI Generated SQL Query\n-- Create date: 30/06/2025\n-- Description:\tIn this hypothetical example, each product in the `Product` table can have multiple tags stored as JSON data in the `ProductTag` table. You can modify the table structure and relationships based on your specific requirements and relationships with existing tables in the schema.\n-- =============================================\n'

In [20]:
query_string

'CREATE TABLE dbo.ProductTag (\n    ProductTagID int PRIMARY KEY,\n    ProductID int,\n    TagData nvarchar(max),\n    CONSTRAINT FK_ProductTag_ProductID FOREIGN KEY (ProductID) REFERENCES dbo.Product(ProductID)\n);'

In [21]:
def find_latest_script_number(folder_path):
    folder = Path(folder_path)
    max_number = 0

    # Pattern to match filenames like Script-<number>.sql
    pattern = re.compile(r"Script-(\d+)\.sql", re.IGNORECASE)

    for file in folder.glob("Script-*.sql"):
        match = pattern.match(file.name)
        if match:
            number = int(match.group(1))
            if number > max_number:
                max_number = number

    return max_number

In [22]:
folder_path = script_dir / "sql_script"
latest_number = find_latest_script_number(folder_path)
print("Latest script number:", latest_number)

Latest script number: 3


In [23]:
output_path = script_dir / "sql_script" / f"Script-{latest_number+1}.sql"

In [24]:
# Make sure the directory exists, create if not
output_path.parent.mkdir(parents=True, exist_ok=True)

In [25]:
full_sql = header + "\n" + query_string

# Write to file
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_sql)

print(f"SQL file saved to: {output_path}")

SQL file saved to: c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester\sql_script\Script-4.sql
